In [1]:
# ! rm -rf PyTorch-Architectures/
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 426, done.
remote: Counting objects: 100% (426/426), done.
remote: Compressing objects: 100% (263/263), done.
remote: Total 857 (delta 233), reused 312 (delta 138), pack-reused 431
Receiving objects: 100% (857/857), 8.43 MiB | 12.51 MiB/s, done.
Resolving deltas: 100% (509/509), done.


In [2]:
%cd PyTorch-Architectures/modeling_xlm/

/kaggle/working/PyTorch-Architectures/modeling_xlm


In [3]:
! pip install datasets
! pip install transformers

     |████████████████████████████████| 147 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 17.7 MB 23.5 MB/s eta 0:00:01   |███████████████▌                | 8.6 MB 4.5 MB/s eta 0:00:03     |█████████████████████████▍      | 14.0 MB 4.5 MB/s eta 0:00:01     |███████████████████████████▋    | 15.3 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 45.0 MB/s eta 0:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 0.16.0
    Uninstalling pyarrow-0.16.0:
      Successfully uninstalled pyarrow-0.16.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import XLMTokenizer
from config_xlm import XLMConfig
from model import XLMWithLMHeadModel
from datasets import load_dataset
from transformers import top_k_top_p_filtering

dataset = load_dataset('cnn_dailymail', '3.0.0')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602. Subsequent calls will reuse this data.


In [5]:
# Defining the model and tokenizer
config = XLMConfig()
config.n_layers = 6
tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
def _init_weights(module):
    """ Initialize the weights. """
    if isinstance(module, nn.Embedding):
        nn.init.normal_(module.weight, mean=0, std=2048 ** -0.5)
    if isinstance(module, nn.Linear):
        nn.init.normal_(module.weight, mean=0, std=0.02)
        if hasattr(module, "bias") and module.bias is not None:
            nn.init.constant_(module.bias, 0.0)
    if isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)
model = XLMWithLMHeadModel(config)
model.apply(_init_weights)
model = model.to(device)

In [6]:
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters = ", total_params)

Total Parameters =  50076097


In [7]:
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, seq_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.seq_length = seq_length
        self.train_list = []
        self.build()

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, index):
        ids = self.train_list[index]['input_ids']
        mask = self.train_list[index]['attention_mask']

        return{
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(ids, dtype=torch.long),
                }

    def build(self):
        for t in self.texts:
            self.train_list.append(tokenizer(t, max_length=self.seq_length, pad_to_max_length=True, truncation=True))

In [8]:
texts_train = dataset['train']['article'][:10000]

texts_valid = dataset['validation']['article'][:1000]

start_time = time.time()
train_dataset = CustomDataset(texts_train, tokenizer, seq_length=512)
valid_dataset = CustomDataset(texts_valid, tokenizer)
torch.save(train_dataset, 'train_dataset.pt')
torch.save(valid_dataset, 'valid_dataset.pt')
print("Dataset Conversion Done!!")
print("Time Taken = ", (time.time() - start_time)/60)

Dataset Conversion Done!!
Time Taken =  5.156346150239309


In [9]:
BATCH_SIZE = 8
LR = 3e-05
EPOCHS = 15

train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=BATCH_SIZE)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=BATCH_SIZE)
print("Length of Train DataLoader: ", len(train_loader))
print("Length of Valid DataLoader: ", len(valid_loader))

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

Length of Train DataLoader:  1250
Length of Valid DataLoader:  125


In [10]:
def compute_loss(model, data_loader, device):
  total_loss = 0
  model.eval()
  with torch.set_grad_enabled(False):
    for sample in data_loader:
      ids = sample['ids'].to(device)
      mask = sample['mask'].to(device)
      labels = sample['ids'].to(device)

      outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
      loss = outputs[0]
      total_loss += loss.item()
  return (total_loss / len(data_loader))

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, sample in enumerate(train_loader):
    ids = sample['ids'].to(device)
    mask = sample['mask'].to(device)
    labels = sample['ids'].to(device)
    
    optimizer.zero_grad()
    
    logits = model(input_ids=ids, attention_mask=mask, labels=labels)
    loss = logits[0]

    # LOGGING
    if idx % 100 == 0:
      print("Batch: %04d/%04d || Epoch: %03d/%03d" % (idx, len(train_loader), epoch+1, EPOCHS))

    loss.backward()
    optimizer.step()

  model.eval()
  with torch.set_grad_enabled(False):
    train_loss = compute_loss(model, train_loader, device)
    valid_loss = compute_loss(model, valid_loader, device)
    print("Train Loss: %.3f" % (train_loss))
    print("Valid Loss: %.3f" % (valid_loss))
  elapsed_epoch_time = (time.time() - start_time) / 60
  print("Epoch Elapsed Time: %d mins" % (elapsed_epoch_time))
total_training_time = (time.time() - start_time) / 60
print("Total Training Time: %d mins" % (elapsed_epoch_time))

Batch: 0000/1250 || Epoch: 001/015
Batch: 0100/1250 || Epoch: 001/015
Batch: 0200/1250 || Epoch: 001/015
Batch: 0300/1250 || Epoch: 001/015
Batch: 0400/1250 || Epoch: 001/015
Batch: 0500/1250 || Epoch: 001/015
Batch: 0600/1250 || Epoch: 001/015
Batch: 0700/1250 || Epoch: 001/015
Batch: 0800/1250 || Epoch: 001/015
Batch: 0900/1250 || Epoch: 001/015
Batch: 1000/1250 || Epoch: 001/015
Batch: 1100/1250 || Epoch: 001/015
Batch: 1200/1250 || Epoch: 001/015
Train Loss: 480.469
Valid Loss: 494.384
Epoch Elapsed Time: 5 mins
Batch: 0000/1250 || Epoch: 002/015
Batch: 0100/1250 || Epoch: 002/015
Batch: 0200/1250 || Epoch: 002/015
Batch: 0300/1250 || Epoch: 002/015
Batch: 0400/1250 || Epoch: 002/015
Batch: 0500/1250 || Epoch: 002/015
Batch: 0600/1250 || Epoch: 002/015
Batch: 0700/1250 || Epoch: 002/015
Batch: 0800/1250 || Epoch: 002/015
Batch: 0900/1250 || Epoch: 002/015
Batch: 1000/1250 || Epoch: 002/015
Batch: 1100/1250 || Epoch: 002/015
Batch: 1200/1250 || Epoch: 002/015
Train Loss: 299.359
Val

In [18]:
# Testing the language model
# text = "he is"
text = "how"
input_ids = tokenizer.encode(text, return_tensors='pt')
mask = torch.ones(input_ids.shape)
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
model.eval()
with torch.set_grad_enabled(False):

  outputs = model(input_ids=input_ids.to(device), attention_mask=mask.to(device))
  logits = outputs[0]
  logits = logits[:, -1, :]
  filtered_next_token_logits = top_k_top_p_filtering(logits, top_k=5, top_p=1.0)
  probs = F.softmax(filtered_next_token_logits, dim=-1)
  next_token = torch.multinomial(probs, num_samples=1)
  generated = torch.cat([input_ids.to(device), next_token], dim=-1)

  resulting_string = tokenizer.decode(generated.tolist()[0], skip_special_tokens=True)
  print(resulting_string)

how but


In [12]:
# Final Notebook --> For the casual modeling objective it still needs a lot of training